In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist

# Load MNIST dataset (you can replace this with your own dataset)
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize and flatten the images
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Define VAE architecture
original_dim = 784
latent_dim = 2

# Encoder
input_img = Input(shape=(original_dim,))
encoded = Dense(256, activation='relu')(input_img)
z_mean = Dense(latent_dim)(encoded)
z_log_var = Dense(latent_dim)(encoded)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = Input(K.int_shape(z)[1:])
decoded = Dense(256, activation='relu')(decoder_input)
decoded_output = Dense(original_dim, activation='sigmoid')(decoded)

# Define models
encoder = Model(input_img, z_mean)

decoder = Model(decoder_input, decoded_output)

# Define VAE model
vae_output = decoder(z)
vae = Model(input_img, vae_output)

# Define loss
reconstruction_loss = K.sum(K.binary_crossentropy(input_img, vae_output), axis=-1)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(x_train, epochs=50, batch_size=128, validation_data=(x_test, None))

# Generate new images
z_sample = np.random.normal(size=(10, latent_dim))
generated_images = decoder.predict(z_sample)

# Display generated images
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(generated_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
